In [83]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from geopy.geocoders import Nominatim
from tqdm import tqdm
from geopy.distance import geodesic
import folium
from folium.plugins import MarkerCluster
import math
import datetime
import urllib.request
import requests
import json


In [130]:
incs = pd.read_csv(r'Data_bomen/Incidenten_oorspronkelijk_volledig.csv')
incs = incs[incs['Damage_Type'] == 'Tree']

incs['Date_time'] = pd.to_datetime(incs['Date'] + ' ' + incs['Incident_Starttime'])
incs['Date_time'] = incs['Date_time'].dt.strftime('%m/%d/%Y %H:%M')
incs = incs[['Incident_ID', 'Date_time', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 
             'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT']]

incs

,Incident_ID,Date_time,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,Damage_Type,LON,LAT
0,511,09/07/2018 08:25,10:00:56,01:35:38,2.0,Amstelveen,Amstelveen,Tree,4.838685,52.281552
1,738,09/10/2018 16:46,18:00:55,01:14:17,2.0,Victor,Amsterdam,Tree,4.930968,52.359724
4,1493,09/21/2018 06:59,08:21:25,01:22:20,2.0,Amstelveen,Amstelveen,Tree,4.879741,52.301365
6,1502,09/21/2018 08:29,10:06:43,01:37:21,2.0,Dirk,Amsterdam,Tree,4.870656,52.355189
7,1507,09/21/2018 09:25,11:43:54,02:17:59,2.0,Teunis,Amsterdam,Tree,4.832333,52.380748
...,...,...,...,...,...,...,...,...,...,...
5729,443469,11/02/2023 19:06,19:06:38,00:00:31,2.0,Weesp,Amsterdam,Tree,5.076108,52.299105
5731,443472,11/02/2023 19:45,20:27:36,00:42:19,2.0,Willem,Amsterdam,Tree,4.908217,52.326775
5732,443474,11/02/2023 19:53,23:41:51,03:48:49,2.0,Pieter,Amsterdam,Tree,4.822130,52.342331
5733,443476,11/02/2023 20:00,20:52:15,00:51:56,2.0,Osdorp,Amsterdam,Tree,4.787237,52.362777


In [131]:
incs['Date_time'].min(), incs['Date_time'].max()

('01/01/2007 13:30', '12/31/2022 12:01')

In [106]:
api_key = '2UQRMKGRVGY6ZZ4MSQB8BBNR6'

url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Amsterdam'
params = {
    'unitGroup': 'us',
    'key': api_key,
    'contentType': 'json',
    'aggregateMinutes': 15,
    'includeAstronomy': False,
    'collectStationContributions': False,
    'startDateTime': '2005-04-07T12:50:00',
    'endDateTime': '2023-11-02T20:25:00'
}

response = requests.get(url, params=params)

if response.status_code == 200:
    weather_data = response.json()
else:
    print(f"Error: {response.status_code}, {response.text}")


In [145]:
weather_api_key = '2UQRMKGRVGY6ZZ4MSQB8BBNR6'
weather_api_endpoint = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Amsterdam'

records = []
labels = [
    "Incident_ID", "Date_time", "Incident_Endtime",
    "Incident_Duration", "Incident_Priority", "Service_Area", "Municipality",
    "Damage_Type", "LON", "LAT", 'temp','precip','wspd'
]
for i, row in incs.iterrows():
    latitude = row['LAT']
    longitude = row['LON']
    IncidentID = row['Incident_ID']
    DamageType = row['Damage_Type']
    datetimeStr = row['Date_time']

    if math.isnan(latitude) or math.isnan(longitude) or (latitude==0 and longitude==0):
        print("Bad latlon {},{}".format(latitude,longitude))
        continue

    if datetimeStr < incs['Date_time'].min():
        print("Bad date {}".format(datetimeStr))
        continue

    try:
        date_time=datetime.datetime.strptime(datetimeStr, '%m/%d/%Y %H:%M')
    except ValueError:
        print("Bad date format {}".format(datetimeStr))
        continue

    latitude='{:.5f}'.format(latitude)
    longitude='{:.5f}'.format(longitude)

    query_params = '&contentType=json&aggregateMinutes=15&unitGroup=us&includeAstronomy=true&collectStationContributions=true&key={}&startDateTime={}&endDateTime={}&locations={},{}'
    query_params=query_params.format(weather_api_key, date_time.isoformat(), date_time.isoformat(),latitude,longitude)
    
    try:
        response = urllib.request.urlopen(weather_api_endpoint+"?"+query_params)
        data = response.read()
    except Exception:
        print("Error reading from {}".format(weather_api_endpoint+"?"+query_params))
        continue
        
    weatherData = json.loads(data.decode('utf-8')) 
    errorCode=weatherData["errorCode"] if 'errorCode' in weatherData else 0
        
    if (errorCode>0):
        print("Error reading from errorCode {}, error={}"
                    .format(weather_api_endpoint+"?"+query_params, errorCode))
        continue 

Error reading from https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Amsterdam?&contentType=json&aggregateMinutes=15&unitGroup=us&includeAstronomy=true&collectStationContributions=true&key=2UQRMKGRVGY6ZZ4MSQB8BBNR6&startDateTime=2018-09-07T08:25:00&endDateTime=2018-09-07T08:25:00&locations=52.28155,4.83869
Error reading from https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Amsterdam?&contentType=json&aggregateMinutes=15&unitGroup=us&includeAstronomy=true&collectStationContributions=true&key=2UQRMKGRVGY6ZZ4MSQB8BBNR6&startDateTime=2018-09-10T16:46:00&endDateTime=2018-09-10T16:46:00&locations=52.35972,4.93097
Error reading from https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Amsterdam?&contentType=json&aggregateMinutes=15&unitGroup=us&includeAstronomy=true&collectStationContributions=true&key=2UQRMKGRVGY6ZZ4MSQB8BBNR6&startDateTime=2018-09-21T06:59:00&endDateTime=2018-09-21T06:59:00&l

KeyboardInterrupt: 

In [146]:
urllib.request.urlopen(weather_api_endpoint+"?"+query_params)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>

In [ ]:
locations = weatherData["locations"]

for locationid in locations:
    location = locations[locationid]

    for value in location["values"]:
        records.append((
            date_time.isoformat(),
            latitude,
            longitude,
            IncidentID,
            DamageType,
            value["temp"],
            value["precip"],
            value["wspd"]
        ))

In [ ]:
output_df = pd.DataFrame.from_records(records, columns=labels)
output_df.to_csv('output.csv', index=False) 

In [141]:
# weather_api_key = '2UQRMKGRVGY6ZZ4MSQB8BBNR6'
# weather_api_endpoint = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Amsterdam'

# combined_data = []
# labels = [
#     "Incident_ID", "Date_time", "Incident_Endtime",
#     "Incident_Duration", "Incident_Priority", "Service_Area", "Municipality",
#     "Damage_Type", "LON", "LAT", 'temp','precip','wspd'
# ]
# for i, row in incs.iterrows():
#     latitude = row['LAT']
#     longitude = row['LON']
#     IncidentID = row['Incident_ID']
#     DamageType = row['Damage_Type']
#     datetimeStr = row['Date_time']

#     if math.isnan(latitude) or math.isnan(longitude) or (latitude==0 and longitude==0):
#         print("Bad latlon {},{}".format(latitude,longitude))
#         continue

#     if datetimeStr < incs['Date_time'].min():
#         print("Bad date {}".format(datetimeStr))
#         continue

#     try:
#         date_time=datetime.datetime.strptime(datetimeStr, '%m/%d/%Y %H:%M')
#     except ValueError:
#         print("Bad date format {}".format(datetimeStr))
#         continue

#     latitude='{:.5f}'.format(latitude)
#     longitude='{:.5f}'.format(longitude)


#     query_params = '&contentType=json&aggregateMinutes=15&unitGroup=us&includeAstronomy=false&collectStationContributions=true&key={}&startDateTime={}&endDateTime={}&locations={},{}'
#     query_params = query_params.format(weather_api_key,date_time.isoformat(), date_time.isoformat(), latitude, longitude)

#     response = urllib.request.urlopen(weather_api_endpoint + "?" + query_params)
#     data = response.read()

#     weather_data = json.loads(data)

#     temperature = weather_data['temperature']
#     precipitation = weather_data['precipitation']
#     windspeed = weather_data['windspeed']

#     combined_entry = {
#         "Incident_ID": IncidentID,
#         "Date_time": datetimeStr,
#         "Incident_Endtime": row['Incident_Endtime'],
#         "Incident_Duration": row['Incident_Duration'],
#         "Incident_Priority": row['Incident_Priority'],
#         "Service_Area": row['Service_Area'],
#         "Municipality": row['Municipality'],
#         "Damage_Type": DamageType,
#         "LON": longitude,
#         "LAT": latitude,
#         "temp": temperature,
#         "precip": precipitation,
#         "wspd": windspeed
#     }

#     combined_data.append(combined_entry)

# combined_df = pd.DataFrame(combined_data, columns=labels)
# combined_df.to_csv('combined_data.csv', index=False)


In [8]:
incidenten = pd.read_csv(r'data_bomen/Stormdata & FireStations geadresseerd.csv')

In [15]:
incidenten_sorted = incidenten[incidenten['Damage_Type'] == 'Tree']
incidenten_sorted = incidenten_sorted.drop('Unnamed: 0', axis=1)

In [16]:
incidenten_sorted

,Incident_ID,Incident_Starttime,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Damage_Type,Municipality,LON,LAT,Day,Month,Year,City,Suburb,Residential,Road,Zipcode
0,511,08:25:18,10:00:56,95.633,2.0,Amstelveen,Tree,Amstelveen,4.838685,52.281552,7,9,2018,NaN,NaN,NaN,Bouwerij,1185 XW
1,738,16:46:38,18:00:55,74.283,2.0,Victor,Tree,Amsterdam,4.930968,52.359724,10,9,2018,Amsterdam,Oost,Dapperbuurt,Linnaeusplantsoen,1093 KL
4,1493,06:59:05,08:21:25,82.333,2.0,Amstelveen,Tree,Amstelveen,4.879741,52.301365,21,9,2018,NaN,NaN,NaN,Oranjebaan,1183 PL
5,1502,08:29:22,10:06:43,97.350,2.0,Dirk,Tree,Amsterdam,4.870656,52.355189,21,9,2018,Amsterdam,Zuid,Museumkwartier,Van Breestraat,1071 ZM
6,1507,09:25:55,11:43:54,137.983,2.0,Teunis,Tree,Amsterdam,4.832333,52.380748,21,9,2018,Amsterdam,Nieuw-West,NaN,M.C. Addicksstraat,1063 VZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5276,436479,18:47:00,20:36:09,109.150,5.0,Victor,Tree,Amsterdam,4.944491,52.352064,31,7,2023,Amsterdam,Oost,Middenmeer,Rutherfordstraat,1098 TM
5277,436603,14:10:34,15:13:30,62.933,2.0,Osdorp,Tree,Amsterdam,4.824533,52.376163,2,8,2023,Amsterdam,Nieuw-West,NaN,Louis Couperusstraat,1064 CE
5282,436834,03:38:27,06:29:11,170.733,5.0,Nico,Tree,Amsterdam,4.928197,52.390568,6,8,2023,Amsterdam,Noord,Tuindorp Buiksloot,Nieuwendammerdijk,1025 LP
5283,436904,21:52:41,22:16:29,23.800,2.0,Nico,Tree,Amsterdam,4.917106,52.368343,6,8,2023,Amsterdam,Centrum,NaN,Hoogte Kadijk,1018 BV


In [60]:
# Create a map centered around Amsterdam
amsterdam_map = folium.Map(location=[52.3676, 4.9041], zoom_start=12)

# Create a MarkerCluster to add incidents to the map
marker_cluster = MarkerCluster().add_to(amsterdam_map)

# Add markers for each incident
for index, incident in incidenten_sorted.iterrows():
    # Extract incident details
    incident_id = incident['Incident_ID']
    lat = incident['LAT']
    lon = incident['LON']
    damage_type = incident['Damage_Type']
    priority = incident['Incident_Priority']

    # Create a popup for the marker
    popup_text = f"Incident ID: {incident_id}<br>Damage Type: {damage_type}<br>Priority: {priority}"

    # Add marker to the cluster
    folium.Marker(location=[lat, lon], popup=popup_text).add_to(marker_cluster)

# Save the map as an HTML file
amsterdam_map.save("incident_map.html")


In [46]:
trees = pd.read_csv(r'BOMEN_DATA_WITH_ZIP.csv')
trees = trees.drop('Unnamed: 0', axis=1)
trees.head()

,OBJECTNUMMER,id,soortnaam,soortnaamKort,SoortnaamNL,soortnaamTop,boomhoogteklasseActueel,stamdiameterklasse,jaarVanAanleg,typeObject,...,typeBeheerderPlus,typeEigenaarPlus,SDVIEW,RADIUS,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Zipcode,Zip4
0,1,919934,Ulmus hollandica `Vegeta`,Ulmus,Huntingdon-iep,Iep (Ulmus),b. 6 tot 9 m.,NaN,1978,Boom niet vrij uitgroeiend,...,Stadsdeel Zuid,Gemeente Amsterdam,K,2,POINT(4.9026919 52.340093),POINT(52.340093 4.9026919),4.902692,52.340093,1185 XW,1185
1,2,919935,Fraxinus excelsior `Westhof`s Glorie`,Fraxinus,Gewone es (cultuurvariëteit),Es (Fraxinus),c. 9 tot 12 m.,"0,2 tot 0,3 m.",1990,Boom niet vrij uitgroeiend,...,Stadsdeel Zuid,Gemeente Amsterdam,K,3,POINT(4.8552081 52.331984),POINT(52.331984 4.8552081),4.855208,52.331984,1185 XW,1185
2,3,919936,Ulmus glabra `Lutescens`,Ulmus,Gele bergiep (cultuurvariëteit),Iep (Ulmus),b. 6 tot 9 m.,NaN,2002,Boom niet vrij uitgroeiend,...,Stadsdeel Zuid,Gemeente Amsterdam,K,2,POINT(4.9036699 52.348837),POINT(52.348837 4.9036699),4.903670,52.348837,1185 XW,1185
3,4,919937,Quercus robur,Quercus,Zomereik,Eik (Quercus),b. 6 tot 9 m.,NaN,1985,Boom niet vrij uitgroeiend,...,Stadsdeel Zuid,Gemeente Amsterdam,K,2,POINT(4.875886 52.341056),POINT(52.341056 4.875886),4.875886,52.341056,1185 XW,1185
4,5,919939,Styphnolobium japonicum,Overig,Honingboom,Styphnolobium,a. tot 6 m.,NaN,2002,Boom niet vrij uitgroeiend,...,Stadsdeel Zuid,Gemeente Amsterdam,K,1,POINT(4.8803689 52.332891),POINT(52.332891 4.8803689),4.880369,52.332891,1185 XW,1185
